In [20]:
import pygame
import random

pygame.init()

WINDOW_WIDTH = 1000
WINDOW_HEIGHT = 1000
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Intersection Simulation")

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0, 100)
GREEN = (0, 255, 0, 100)
YELLOW = (255, 255, 0, 100)
GRAY = (60, 60, 60)

ambient_1 = pygame.image.load('assets/ambient_1.png')
ambient_2 = pygame.image.load('assets/ambient_2.png')
ambient_3 = pygame.image.load('assets/ambient_3.png')
ambient_4 = pygame.image.load('assets/ambient_4.png')

# Set audio effect in loop
pygame.mixer.music.load('assets/Street Sound Effect.mp3')
pygame.mixer.music.play(-1)

# Load image, resize it to be 370x370 and put it in the right corner
ambient_1 = pygame.transform.scale(ambient_1, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_2 = pygame.transform.scale(ambient_2, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_3 = pygame.transform.scale(ambient_3, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_4 = pygame.transform.scale(ambient_4, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
window.blit(ambient_1, (0, 0))
window.blit(ambient_2, (WINDOW_WIDTH//2 + 30, 0))
window.blit(ambient_3, (WINDOW_WIDTH//2 + 30, WINDOW_HEIGHT//2 + 30))
window.blit(ambient_4, (0, WINDOW_HEIGHT//2 + 30))

class Car:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.speed = 2
        self.width = 20
        self.length = 40
        self.stopped = False
        self.color = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
        self.turn_right = random.choice([False, True])
        self.waiting_time = 0

    def move(self):
        if not self.stopped:
            if self.direction == 'up':
                self.y -= self.speed
            elif self.direction == 'down':
                self.y += self.speed
            elif self.direction == 'left':
                self.x -= self.speed
            elif self.direction == 'right':
                self.x += self.speed

    def draw(self, win, pygame):
        if self.direction == 'up':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.length))
            # Check if car wants to turn, if so add a blinking turn signal
            if self.turn_right == True and pygame.time.get_ticks()//1000 % 2 == 0:
                pygame.draw.polygon(win, (255, 85, 0), [(self.x + 20, self.y), (self.x + 20, self.y + 10), (self.x + 25, self.y - 5), (self.x + 25, self.y + 15)])
            # Add waiting_time text to the car that moves with it
            font = pygame.font.Font(None, 20)
            text = font.render(str(self.waiting_time), True, (255, 255, 255))
            text = pygame.transform.rotate(text, 90)
            win.blit(text, (self.x + 5, self.y + 5))
        elif self.direction == 'down':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.length))
            # Check if car wants to turn, if so add a blinking turn signal
            if self.turn_right == True and pygame.time.get_ticks()//1000 % 2 == 0:
                pygame.draw.polygon(win, (255, 85, 0), [(self.x, self.y + 30), (self.x, self.y + 40), (self.x - 5, self.y + 25), (self.x - 5, self.y + 45)])
            # Add waiting_time text to the car that moves with it
            font = pygame.font.Font(None, 20)
            text = font.render(str(self.waiting_time), True, (255, 255, 255))
            text = pygame.transform.rotate(text, 90)
            win.blit(text, (self.x + 5, self.y + 5))
        elif self.direction == 'left':
            pygame.draw.rect(win, self.color, (self.x, self.y, self.length, self.width))
            # Check if car wants to turn, if so add a blinking turn signal
            if self.turn_right == True and pygame.time.get_ticks()//1000 % 2 == 0:
                pygame.draw.polygon(win, (255, 85, 0), [(self.x, self.y), (self.x + 10, self.y), (self.x - 5, self.y - 5), (self.x + 15, self.y - 5)])
            # Add waiting_time text to the car that moves with it
            font = pygame.font.Font(None, 20)
            text = font.render(str(self.waiting_time), True, (255, 255, 255))
            win.blit(text, (self.x + 5, self.y + 5))
        else:
            pygame.draw.rect(win, self.color, (self.x, self.y, self.length, self.width))
            # Check if car wants to turn, if so add a blinking turn signal
            if self.turn_right == True and pygame.time.get_ticks()//1000 % 2 == 0:
                pygame.draw.polygon(win, (255, 85, 0), [(self.x, self.y + 20), (self.x, self.y + 20), (self.x - 5, self.y + 25), (self.x - 5, self.y + 45)])
            # Add waiting_time text to the car that moves with it
            font = pygame.font.Font(None, 20)
            text = font.render(str(self.waiting_time), True, (255, 255, 255))
            win.blit(text, (self.x + 5, self.y + 5))

    def stop(self):
        self.stopped = True

    def can_move(self, other_cars):
        for other_car in other_cars:
            if other_car.stopped:
                if self.direction == other_car.direction == 'up':
                    if other_car.y + other_car.length + 4 <= self.y <= other_car.y + other_car.length + 6:
                        return False
                elif self.direction == other_car.direction == 'down':
                    if other_car.y - self.length - 6 <= self.y <= other_car.y - self.length - 4:
                        return False
                elif self.direction == other_car.direction == 'left':
                    if other_car.x + other_car.length + 4 <= self.x <= other_car.x + other_car.length + 6:
                        return False
                elif self.direction == other_car.direction == 'right':
                    if other_car.x - self.length - 6 <= self.x <= other_car.x - self.length - 4:
                        return False
        return True

    def turn_or_straight(self):
        if self.turn_right == True:
            if self.direction == 'up' and self.y <= WINDOW_HEIGHT//2:
                self.direction = 'right'
                self.x += self.length // 2
                self.y = WINDOW_HEIGHT//2 + 5
                self.turn_right = False
            elif self.direction == 'down' and self.y + self.length >= WINDOW_HEIGHT//2:
                self.direction = 'left'
                self.x -= self.length // 2
                self.y = WINDOW_HEIGHT//2 - 20 - 4
                self.turn_right = False
            elif self.direction == 'left' and self.x <= WINDOW_WIDTH//2:
                self.direction = 'up'
                self.x = WINDOW_WIDTH//2 + 5
                self.y -= self.length // 2
                self.turn_right = False
            elif self.direction == 'right' and self.x + self.length >= WINDOW_WIDTH//2:
                self.direction = 'down'
                self.x = WINDOW_WIDTH//2 - 20 - 4
                self.y += self.length // 2
                self.turn_right = False

class Stoplight:
    def __init__(self, color):
        self.color_NS = color
        if self.color_NS == GREEN:
            self.color_EW = RED
        else:
            self.color_EW = GREEN
        self.time_yellow = 0
        self.time_green = 0

    def switch_yellow(self):
        if self.color_NS == GREEN:
            self.color_NS = YELLOW
        elif self.color_EW == GREEN:
            self.color_EW = YELLOW

    def update_stoplight(self):
        if self.color_NS == GREEN or self.color_EW == GREEN:
            self.time_green += 1
        if self.color_NS == YELLOW or self.color_EW == YELLOW:
            self.time_yellow += 1

        if self.time_yellow >= 90:
            if self.color_NS == YELLOW:
                self.color_NS = RED
                self.color_EW = GREEN
                self.time_green = 0
            elif self.color_EW == YELLOW:
                self.color_EW = RED
                self.color_NS = GREEN
                self.time_green = 0
            self.time_yellow = 0
        
        

def main():
    run = True
    clock = pygame.time.Clock()

    cars = []
    prev_time = 0
    stoplight = Stoplight(random.choice([GREEN, RED]))

    while run:
        clock.tick(30)
        stoplight.update_stoplight()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        time = pygame.time.get_ticks() // 1000
        if time != prev_time:
            if time % 2 == 0:
                direction = random.choice(['up', 'down', 'left', 'right'])
                if direction == 'up':
                    cars.append(Car(WINDOW_WIDTH//2 + 5, WINDOW_HEIGHT, direction))
                elif direction == 'down':
                    cars.append(Car(WINDOW_WIDTH//2 - 20 - 4, 0, direction))
                elif direction == 'left':
                    cars.append(Car(WINDOW_WIDTH, WINDOW_HEIGHT//2 - 20 - 4, direction))
                elif direction == 'right':
                    cars.append(Car(0, WINDOW_HEIGHT//2 + 5, direction))
            prev_time = time

        if stoplight.time_green >= 300:
            stoplight.switch_yellow()
        
        for car in cars:
            if not car.stopped:
                if ((car.direction == 'up' and car.y == WINDOW_HEIGHT//2 + 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                    (car.direction == 'down' and car.y + car.length == WINDOW_HEIGHT//2 - 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                    (car.direction == 'left' and car.x == WINDOW_WIDTH//2 + 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW)) or
                    (car.direction == 'right' and car.x + car.length == WINDOW_WIDTH//2 - 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW))) or not car.can_move(cars):
                    car.stop()
                else:
                    if ((car.direction == 'up' and car.y <= WINDOW_HEIGHT//2 + car.speed and car.y >= WINDOW_HEIGHT//2 - car.speed) or
                        (car.direction == 'down' and car.y + car.length >= WINDOW_HEIGHT//2 - car.speed and car.y + car.length <= WINDOW_HEIGHT//2 + car.speed) or
                        (car.direction == 'left' and car.x <= WINDOW_WIDTH//2 + car.speed and car.x >= WINDOW_WIDTH//2 - car.speed) or
                        (car.direction == 'right' and car.x + car.length >= WINDOW_WIDTH//2 - car.speed and car.x + car.length <= WINDOW_WIDTH//2 + car.speed)):
                        car.turn_or_straight()
                    car.move()
            elif car.stopped and ((car.direction in ['up', 'down'] and stoplight.color_NS == GREEN) or (car.direction in ['left', 'right'] and stoplight.color_EW == GREEN)):
                car.stopped = False
                car.move()
            if car.stopped:
                car.waiting_time += 1
            else:
                car.waiting_time = 0

        cars = [car for car in cars if 0 <= car.x <= WINDOW_WIDTH and 0 <= car.y <= WINDOW_HEIGHT]

        # Draw intersection
        ## Draw road
        pygame.draw.line(window, GRAY, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 60)
        pygame.draw.line(window, GRAY, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 60)
        ## Draw lanes
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 - 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 - 28), 1)
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 + 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 + 28), 1)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 28, 0), (WINDOW_WIDTH//2 - 28, WINDOW_HEIGHT), 1)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 28, 0), (WINDOW_WIDTH//2 + 28, WINDOW_HEIGHT), 1)
        pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 4)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 4)
        ## Draw crosswalks
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 -200), (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 - 180), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 23), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 23), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 17), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 17), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 12), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 12), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 6), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 6), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 6), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 6), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 12), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 12), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 17), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 17), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 23), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 23), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 23), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 23), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 17), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 17), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 12), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 12), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 6), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 6), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 6), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 6), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 12), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 12), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 17), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 17), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 23), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 23), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 + 220), 2)
        pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 + 220), 2)
        ## Cover intersection
        pygame.draw.rect(window, GRAY, (WINDOW_WIDTH//2 - 29, WINDOW_HEIGHT//2 - 29, 60, 60))
        ## Draw stoplights as stop lines
        pygame.draw.line(window, stoplight.color_NS, (WINDOW_WIDTH//2 - 27, WINDOW_HEIGHT//2 - 32), (WINDOW_WIDTH//2 - 2, WINDOW_HEIGHT//2 - 32), 5)
        pygame.draw.line(window, stoplight.color_NS, (WINDOW_WIDTH//2 + 3, WINDOW_HEIGHT//2 + 33), (WINDOW_WIDTH//2 + 27, WINDOW_HEIGHT//2 + 33), 5)
        pygame.draw.line(window, stoplight.color_EW, (WINDOW_WIDTH//2 - 32, WINDOW_HEIGHT//2 + 3), (WINDOW_WIDTH//2 - 32, WINDOW_HEIGHT//2 + 27), 5)
        pygame.draw.line(window, stoplight.color_EW, (WINDOW_WIDTH//2 + 33, WINDOW_HEIGHT//2 - 27), (WINDOW_WIDTH//2 + 33, WINDOW_HEIGHT//2 - 2), 5)
        
        # Draw cars
        for car in cars:
            car.draw(window, pygame)
        
        pygame.display.update()

    pygame.quit()

if __name__ == "__main__":
    main()